Data Preprocessing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import gzip
import json
import os
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [1]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz

--2021-09-27 21:45:00--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495854086 (473M) [application/x-gzip]
Saving to: ‘reviews_Electronics_5.json.gz.2’

reviews_Electronics 100%[===================>] 472.88M  3.67MB/s    in 2m 8s   

2021-09-27 21:47:09 (3.68 MB/s) - ‘reviews_Electronics_5.json.gz.2’ saved [495854086/495854086]



In [4]:
### load the meta data

data = []
with gzip.open('reviews_Electronics_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

Checking for null values

In [5]:
df = df.dropna(subset=['reviewText'])

In [6]:
##########################################
# Convert time object to datetime and create a new column named 'time'
##########################################
df['review_time'] = df.reviewTime.str.replace(',', "")
df['review_time'] = pd.to_datetime(df['review_time'], format = '%m %d %Y')

# Drop redundant 'reviewTime' column
df = df.drop('reviewTime', axis = 1)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,review_time
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,2013-06-02
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,2010-11-25
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,2010-09-09
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,2010-11-24
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,2011-09-29


In [7]:
def clean_text(text):
  text = ' '.join(re.sub("<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});", "", text).split())
  #Then tokenisation
  tokens = word_tokenize(text)
  # convert to lower case
  tokens = [w.lower() for w in tokens]
  # remove punctuation from each word
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  # remove remaining tokens that are not alphabetic
  words = [word for word in stripped if word.isalpha()]
  # filter out stop words
  stop_words = set(stopwords.words('english'))
  # You can add more stop words here, specific for texts
  words = [w for w in words if not w in stop_words]
  # stemming of words
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(word) for word in words]
  # Convert from list to a sentence again
  text = ' '.join(word for word in words)
  return text

In [8]:
#Process the segments here
df['reviewText'] = df['reviewText'].apply(clean_text)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,review_time
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",got gps husband otr road trucker impressed shi...,5.0,Gotta have GPS!,1370131200,2013-06-02
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]",professional otr truck driver bought tnd truck...,1.0,Very Disappointed,1290643200,2010-11-25
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]",well say unit truck four day prior garmin nont...,3.0,1st impression,1283990400,2010-09-09
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]",going write long review even thought unit dese...,2.0,"Great grafics, POOR GPS",1290556800,2010-11-24
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",mine year got try route non truck route tellin...,1.0,"Major issues, only excuses for support",1317254400,2011-09-29


In [9]:
df.to_csv("/content/drive/MyDrive/Dataset/Electronics_processed.csv")